In [1]:
import json
import pandas as pd
import nltk

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from matplotlib.colors import ListedColormap
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tag import pos_tag

from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

import spacy
from spacy import displacy
import requests
import time


spacy.cli.download("en_core_web_sm")
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Celin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Celin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Celin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Celin\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [2]:
#getting data from yahoo finance

import yfinance as yf
nasdaq = yf.Ticker("^IXIC")

#getting data for first half of year 2022
df_nasdaq = nasdaq.history(start = "2022-01-01", end = "2022-12-31")

#daily labels

df_labels = pd.DataFrame() 
df_labels["Open-Close"] = df_nasdaq['Open'] - df_nasdaq["Close"]


In [3]:
#removing time of day
df_nasdaq.index = df_nasdaq.index.date

In [4]:
#creating label for each day
df_labels['label'] = 'dummy'
for i in range(len(df_labels)):
    if df_labels['Open-Close'][i] < 0:
        df_labels['label'][i] = 0
    elif df_labels['Open-Close'][i] > 0:
        df_labels['label'][i] = 1

<ipython-input-4-d0fb6ff2ee21>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_labels['label'][i] = 0
<ipython-input-4-d0fb6ff2ee21>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_labels['label'][i] = 1


In [5]:
#label for each day 
y = df_labels['label']

In [6]:
#remove first day
y = y.iloc[1:]

In [7]:
api_key = "wwFVxDThbk6roswX4uS8aA7zC7Akuqt0"
year = 2022
articles = []

#split the request into quarters because of too large requests

for month in range(1, 4):
    url = f"https://api.nytimes.com/svc/archive/v1/{year}/{month}.json?api-key={api_key}"

    response = requests.get(url)
    data = response.json()
    time.sleep(10)

    # Check if the response was successful
    if response.status_code != 200:
        continue

    # Check if 'response' key exists in the data
    if 'response' not in data:
        continue

    articles.extend(data['response']['docs']) 

In [8]:
for month in range(4, 7):
    url = f"https://api.nytimes.com/svc/archive/v1/{year}/{month}.json?api-key={api_key}"

    response = requests.get(url)
    data = response.json()
    time.sleep(10)

    # Check if the response was successful
    if response.status_code != 200:
        continue

    # Check if 'response' key exists in the data
    if 'response' not in data:
        continue

    articles.extend(data['response']['docs']) 

In [9]:
for month in range(7, 10):
    url = f"https://api.nytimes.com/svc/archive/v1/{year}/{month}.json?api-key={api_key}"

    response = requests.get(url)
    data = response.json()
    time.sleep(10)

    # Check if the response was successful
    if response.status_code != 200:
        continue

    # Check if 'response' key exists in the data
    if 'response' not in data:
        continue

    articles.extend(data['response']['docs']) 

In [23]:
for month in range(10, 13):
    url = f"https://api.nytimes.com/svc/archive/v1/{year}/{month}.json?api-key={api_key}"

    response = requests.get(url)
    data = response.json()
    time.sleep(10)

    # Check if the response was successful
    if response.status_code != 200:
        continue

    # Check if 'response' key exists in the data
    if 'response' not in data:
        continue

    articles.extend(data['response']['docs']) 

In [24]:
df = pd.DataFrame.from_records(articles)

In [25]:
df.tail()

,abstract,web_url,snippet,lead_paragraph,print_section,print_page,source,multimedia,headline,keywords,pub_date,document_type,news_desk,section_name,subsection_name,byline,type_of_material,_id,word_count,uri
48702,She infused her work with political and femini...,https://www.nytimes.com/2022/12/31/obituaries/...,She infused her work with political and femini...,"Jean Franco, a pioneering literary scholar who...",NaN,NaN,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","{'main': 'Jean Franco, 98, Pioneering Scholar ...","[{'name': 'subject', 'value': 'Colleges and Un...",2022-12-31T19:10:36+0000,article,Obits,Obituaries,NaN,"{'original': 'By Clay Risen', 'person': [{'fir...",News,nyt://article/4da4f196-7b40-57ee-ad72-2e0c74ef...,969,nyt://article/4da4f196-7b40-57ee-ad72-2e0c74ef...
48703,"Heavy rain, snow and wind were battering parts...",https://www.nytimes.com/2022/12/31/us/californ...,"Heavy rain, snow and wind were battering parts...",Heavy rain and snow caused landslides and floo...,NaN,NaN,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'Extreme Weather in California Causes...,"[{'name': 'subject', 'value': 'Floods', 'rank'...",2022-12-31T19:21:09+0000,article,Express,U.S.,NaN,"{'original': 'By April Rubin', 'person': [{'fi...",News,nyt://article/5bd9694a-20ef-5f9a-9b32-bb1b829a...,530,nyt://article/5bd9694a-20ef-5f9a-9b32-bb1b829a...
48704,"Each year, the comic headlined the club during...",https://www.nytimes.com/2022/12/31/arts/televi...,"Each year, the comic headlined the club during...",How do you honor the death of a comedy club? F...,NaN,NaN,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'Dave Attell Bids a Heartfelt (and Hi...,"[{'name': 'subject', 'value': 'Comedy and Humo...",2022-12-31T20:18:43+0000,article,Culture,Arts,Television,{'original': 'By Jason Zinoman and Photographs...,News,nyt://article/dae0c45e-3db9-5a73-b711-a08e94dc...,1163,nyt://article/dae0c45e-3db9-5a73-b711-a08e94dc...
48705,"What brought the owl to the city of Cypress, i...",https://www.nytimes.com/2022/12/31/us/snowy-ow...,"What brought the owl to the city of Cypress, i...",The forbidding frozen wilderness of the high A...,NaN,NaN,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': '‘Extremely Rare’ Snowy Owl Sighting ...,"[{'name': 'subject', 'value': 'Owls', 'rank': ...",2022-12-31T20:22:56+0000,article,Express,U.S.,NaN,"{'original': 'By Michael Levenson', 'person': ...",News,nyt://article/86e3ee7f-db58-5517-a18b-dbcf78ce...,653,nyt://article/86e3ee7f-db58-5517-a18b-dbcf78ce...
48706,Answering reader queries about the state of th...,https://www.nytimes.com/2022/12/31/arts/music/...,Answering reader queries about the state of th...,"Each year, oodles of questions pour in from th...",NaN,NaN,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'Year-End Listener Mailbag: Your 2022...,"[{'name': 'subject', 'value': 'audio-neutral-i...",2022-12-31T20:46:45+0000,article,Culture,Arts,Music,"{'original': '', 'person': [], 'organization':...",News,nyt://article/f27da0fd-710f-525f-a10a-05c14dfd...,141,nyt://article/f27da0fd-710f-525f-a10a-05c14dfd...


In [26]:
articles_list = []
# Process the articles
for article in articles:
    headline = article['headline']['main']
    lead_paragraph = article['lead_paragraph']
    section_name = article['section_name']
    date = article['pub_date']        
        # Tokenize the words in the lead paragraph
    space_tokenizer = nltk.tokenize.SpaceTokenizer()
    tokenized_paragraph = space_tokenizer.tokenize(lead_paragraph)
        
        # Convert words to lowercase   
    tokenized_paragraph_lower = [w.lower() for w in tokenized_paragraph]
        
        # Remove stopwords from the tokenized paragraph
    stop_words = set(stopwords.words('english'))
    tokens_without_stopwords = [w for w in tokenized_paragraph_lower if w not in stop_words]
    tokens_without_stopwords = [token for token in tokens_without_stopwords if len(token) > 0]
        
        # POS Tag the token
    paragraph_processed = nltk.pos_tag(tokens_without_stopwords)
        
        # Lemmatize the token
    wordnet = nltk.WordNetLemmatizer()
        #tokens_lemmatized = [wordnet.lemmatize(t) for t in tokenized_paragraph_lower]
        
    articles_list.append({
        "headline": headline,
        "lead_paragraph": lead_paragraph,
        "section_name": section_name,
         #   "tokens_lemmad": tokens_lemmatized,
        "processed_paragraph": paragraph_processed,
        'date': date[:10]
        })
        

In [27]:
articles_df = pd.DataFrame.from_records(articles_list)

In [28]:
articles_df = articles_df.drop('section_name', axis=1)

In [29]:
articles_df.tail(5)

,headline,lead_paragraph,processed_paragraph,date
48702,"Jean Franco, 98, Pioneering Scholar of Latin A...","Jean Franco, a pioneering literary scholar who...","[(jean, JJ), (franco,, NN), (pioneering, VBG),...",2022-12-31
48703,Extreme Weather in California Causes Flooding ...,Heavy rain and snow caused landslides and floo...,"[(heavy, JJ), (rain, NN), (snow, NNS), (caused...",2022-12-31
48704,Dave Attell Bids a Heartfelt (and Hilarious) F...,How do you honor the death of a comedy club? F...,"[(honor, JJ), (death, NN), (comedy, NN), (club...",2022-12-31
48705,‘Extremely Rare’ Snowy Owl Sighting Transfixes...,The forbidding frozen wilderness of the high A...,"[(forbidding, VBG), (frozen, JJ), (wilderness,...",2022-12-31
48706,Year-End Listener Mailbag: Your 2022 Questions...,"Each year, oodles of questions pour in from th...","[(year,, NN), (oodles, NNS), (questions, NNS),...",2022-12-31


In [30]:
#grouping headlines and paragraph for each day
grouped_df = articles_df.groupby('date').agg({'headline': lambda x: ' '.join(x), 'lead_paragraph': lambda x: ' '.join(x)})

In [31]:
#transforming index to datetime
from datetime import datetime

dates = []
# Example with the standard date and time format
for i in range(len(grouped_df)):
    date_str = grouped_df.index[i]
    date_format = '%Y-%m-%d'
    date_obj = datetime.strptime(date_str, date_format)
    dates.append(date_obj)

In [32]:
#getting the same date format as in the label
grouped_df.index = dates
grouped_df.index = grouped_df.index.date

In [33]:
#only extract the articles that were published on dates that theres also data for the labels

grouped_df = grouped_df.loc[df_nasdaq.index]

In [34]:
#sentiment scores
#sentiments each get a score between 0 and 1 
analyzer = SentimentIntensityAnalyzer()
#headlines
neg = []
pos = []
neu = []
compound = []
for sentence in grouped_df["headline"]:
    polarity_scores = analyzer.polarity_scores(sentence)
    neg.append(polarity_scores['neg'])
    pos.append(polarity_scores['pos'])
    neu.append(polarity_scores['neu'])
    compound.append(polarity_scores['compound'])

neg = np.array(neg)[..., None]
neu = np.array(neu)[..., None]
pos = np.array(pos)[..., None]
compound = np.array(compound)[..., None]

#create feature matrix for headlines
X_sentiment_detailed_headlines = np.concatenate((neg, neu, pos, compound), axis = 1)

#lead paragraphs

neg_para = []
pos_para = []
neu_para = []
compound_para = []
for sentence in grouped_df["lead_paragraph"]:
    polarity_scores = analyzer.polarity_scores(sentence)
    neg_para.append(polarity_scores['neg'])
    pos_para.append(polarity_scores['pos'])
    neu_para.append(polarity_scores['neu'])
    compound_para.append(polarity_scores['compound'])

neg_para = np.array(neg_para)[..., None]
neu_para = np.array(neu_para)[..., None]
pos_para = np.array(pos_para)[..., None]
compound_para = np.array(compound_para)[..., None]

#create feature matrix for paragraphs
X_sentiment_detailed_paragraphs = np.concatenate((neg_para, neu_para, pos_para, compound_para), axis = 1)

In [35]:
#remove last day
X_sentiment_detailed_headlines = X_sentiment_detailed_headlines[:-1]
X_sentiment_detailed_paragraphs = X_sentiment_detailed_paragraphs[:-1]

In [36]:
X_sentiment_detailed_paragraphs.shape

(250, 4)

In [37]:
#Named entity features 
#headlines
NER = spacy.load("en_core_web_sm")
text1_headlines = [NER(string) for string in grouped_df["headline"]]

In [38]:
#headlines
labels_headlines = []
for text in text1_headlines:
    labels = []
    for word in text.ents:
        labels.append(word.label_)
    labels_headlines.append(' '.join(labels))

In [39]:
count = CountVectorizer()
bag = count.fit_transform(labels_headlines)

In [40]:
bag = bag.toarray()

In [41]:
bag.shape

(251, 18)

In [42]:
#normalized feature matrix 
X_topic_label_feature_headlines = bag/np.max(bag)

In [43]:
#remove last day
X_topic_label_feature_headlines = X_topic_label_feature_headlines[:-1]

In [44]:
#lead paragraph
text1_paragraphs = [NER(string) for string in grouped_df["lead_paragraph"]]

In [45]:
labels_paragraph = []
for text in text1_paragraphs:
    labels = []
    for word in text.ents:
        labels.append(word.label_)
    labels_paragraph.append(' '.join(labels))

In [46]:
bag_paragraphs = count.fit_transform(labels_paragraph).toarray()

In [47]:
#normalized feature matrix
X_topic_label_feature_paragraph = bag_paragraphs/np.max(bag_paragraphs)
X_topic_label_feature_paragraph = X_topic_label_feature_paragraph[:-1]

In [48]:
# concat all features 
df_features = np.concatenate((X_sentiment_detailed_headlines, X_sentiment_detailed_paragraphs, X_topic_label_feature_paragraph, X_topic_label_feature_headlines), axis=1)
df_features.shape

(250, 44)

In [49]:
# get labes from y
y_labels = y.values
y_labels = y_labels.astype(int)
y_labels

array([1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1,
       0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 0, 0])

In [50]:
unique_elements, counts = np.unique(y_labels, return_counts=True)
for element, count in zip(unique_elements, counts):
    print(f"{element}: {count}")

0: 118
1: 132


# Train different models

**Perceptron**

In [58]:
# Create the Perceptron model
ppn = Perceptron(random_state=1)

# Define the parameter grid for grid search
param_grid = {
    'max_iter': [10, 20, 30, 40, 50, 100, 500, 1000],  
    'eta0': [0.00000000000000000000000001, 0.000000000001, 0.0000001, 0.000001, 0.01, 0.05, 0.1, 0.2]    
}

# Perform grid search and evaluation with different train-test splits
for train_size in [70, 75, 80, 85, 90]:
    
    # Split the data into training and testing subsets
    train_size_index = int(df_features.shape[0] * (train_size / 100))
    X_train = df_features[:train_size_index]
    X_test = df_features[train_size_index:]
    y_train = y_labels[:train_size_index]
    y_test = y_labels[train_size_index:]


    # Create the GridSearchCV object
    grid_search = GridSearchCV(ppn, param_grid, cv=5, n_jobs=-1)

    # Perform the grid search on your data
    grid_search.fit(X_train, y_train)

    # Get the best hyperparameters from the grid search
    best_max_iter = grid_search.best_params_['max_iter']
    best_eta = grid_search.best_params_['eta0']

    # Train the final model with the best hyperparameters
    final_ppn = Perceptron(max_iter=best_max_iter, eta0=best_eta, random_state=1)
    final_ppn.fit(X_train, y_train)

    # Evaluate the model on the test set
    y_pred = final_ppn.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    print(f"Train size: {train_size}, Best max_iter: {best_max_iter}, Best eta: {best_eta}, Accuracy: {accuracy:.2f}")


Train size: 70, Best max_iter: 10, Best eta: 1e-26, Accuracy: 0.59
Train size: 75, Best max_iter: 10, Best eta: 1e-26, Accuracy: 0.54
Train size: 80, Best max_iter: 10, Best eta: 1e-26, Accuracy: 0.50
Train size: 85, Best max_iter: 20, Best eta: 0.05, Accuracy: 0.45
Train size: 90, Best max_iter: 10, Best eta: 1e-26, Accuracy: 0.40


**Decision Tree**

In [61]:
# Create the DecisionTreeClassifier model
tree_clf = DecisionTreeClassifier(random_state=1)

# Define the parameter grid for grid search
param_grid = {
    'max_depth': [2, 4, 6, 8],         # Different values for max_depth
    'min_samples_split': [2, 5, 10],   # Different values for min_samples_split
    'min_samples_leaf': [1, 2, 4]      # Different values for min_samples_leaf
}

# Perform grid search and evaluation with different train-test splits
for train_size in [70, 75, 80, 85, 90]:
    # Split the data into training and testing subsets
    train_size_index = int(df_features.shape[0] * (train_size / 100))
    X_train = df_features[:train_size_index]
    X_test = df_features[train_size_index:]
    y_train = y_labels[:train_size_index]
    y_test = y_labels[train_size_index:]


    # Create the GridSearchCV object
    grid_search = GridSearchCV(tree_clf, param_grid, cv=5, n_jobs=-1)

    # Perform the grid search on your data
    grid_search.fit(X_train, y_train)

    # Get the best hyperparameters from the grid search
    best_max_depth = grid_search.best_params_['max_depth']
    best_min_samples_split = grid_search.best_params_['min_samples_split']
    best_min_samples_leaf = grid_search.best_params_['min_samples_leaf']

    # Train the final model with the best hyperparameters
    final_tree_clf = DecisionTreeClassifier(
        max_depth=best_max_depth,
        min_samples_split=best_min_samples_split,
        min_samples_leaf=best_min_samples_leaf,
        random_state=1
    )
    final_tree_clf.fit(X_train, y_train)

    # Evaluate the model on the test set
    accuracy = final_tree_clf.score(X_test, y_test)
    print(f"Train size: {train_size}, Best max_depth: {best_max_depth}, Best min_samples_split: {best_min_samples_split}, Best min_samples_leaf: {best_min_samples_leaf}, Accuracy: {accuracy:.2f}")

Train size: 70, Best max_depth: 8, Best min_samples_split: 2, Best min_samples_leaf: 2, Accuracy: 0.57
Train size: 75, Best max_depth: 6, Best min_samples_split: 2, Best min_samples_leaf: 2, Accuracy: 0.59
Train size: 80, Best max_depth: 4, Best min_samples_split: 2, Best min_samples_leaf: 4, Accuracy: 0.66
Train size: 85, Best max_depth: 6, Best min_samples_split: 10, Best min_samples_leaf: 4, Accuracy: 0.55
Train size: 90, Best max_depth: 6, Best min_samples_split: 2, Best min_samples_leaf: 1, Accuracy: 0.56


**Support Vector Machines**

In [62]:
# Create the SVM model
svm_clf = SVC(random_state=42)

# Define the parameter grid for grid search
param_grid = {
    'kernel': ['linear', 'rbf'],          # Different kernels to try
    'C': [0.001, 0.1, 1, 10],             # Different values of C
    'gamma': ['scale', 'auto', 0.1, 1]    # Different values of gamma (only for 'rbf' kernel)
}

# Perform grid search and evaluation with different train-test splits
for train_size in [70, 75, 80, 85, 90]:
    # Split the data into training and testing subsets
    train_size_index = int(df_features.shape[0] * (train_size / 100))
    X_train = df_features[:train_size_index]
    X_test = df_features[train_size_index:]
    y_train = y_labels[:train_size_index]
    y_test = y_labels[train_size_index:]


    # Create the GridSearchCV object
    grid_search = GridSearchCV(svm_clf, param_grid, cv=5, n_jobs=-1)

    # Perform the grid search on your data
    grid_search.fit(X_train, y_train)

    # Get the best hyperparameters from the grid search
    best_kernel = grid_search.best_params_['kernel']
    best_C = grid_search.best_params_['C']
    best_gamma = grid_search.best_params_['gamma']

    # Train the final model with the best hyperparameters
    final_svm_clf = SVC(kernel=best_kernel, C=best_C, gamma=best_gamma, random_state=42)
    final_svm_clf.fit(X_train, y_train)

    # Make predictions on the test data
    y_pred = final_svm_clf.predict(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Train size: {train_size}, Best kernel: {best_kernel}, Best C: {best_C}, Best gamma: {best_gamma}, Accuracy: {accuracy:.2f}")

Train size: 70, Best kernel: rbf, Best C: 1, Best gamma: auto, Accuracy: 0.40
Train size: 75, Best kernel: linear, Best C: 0.001, Best gamma: scale, Accuracy: 0.57
Train size: 80, Best kernel: linear, Best C: 10, Best gamma: scale, Accuracy: 0.54
Train size: 85, Best kernel: linear, Best C: 0.001, Best gamma: scale, Accuracy: 0.55
Train size: 90, Best kernel: linear, Best C: 0.001, Best gamma: scale, Accuracy: 0.60
